In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler, KMeansSMOTE, SVMSMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# kagglehub.dataset_download("whenamancodes/blood-transfusion-dataset")
transfusion = pd.read_csv("E:\Workings\Thesis\Percentile v SMOTE\Datasets smote\\transfusion.csv")

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
C:\Users\danis\AppData\Local\Temp\ipykernel_9480\1983729316.py:2: SyntaxWarning: invalid escape sequence '\W'
  transfusion = pd.read_csv("E:\Workings\Thesis\Percentile v SMOTE\Datasets smote\\transfusion.csv")


In [3]:
transfusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [4]:
from sklearn.model_selection import train_test_split
from collections import Counter

In [5]:
transfusion['whether he/she donated blood in March 2007'].value_counts()

whether he/she donated blood in March 2007
0    570
1    178
Name: count, dtype: int64

In [6]:
df_resampled, y_resampled = transfusion.copy(), transfusion['whether he/she donated blood in March 2007']
df_resampled2 = df_resampled.copy()     # saving same data to be used for our proposed method 
print("Original class distribution:", Counter(y_resampled))
del df_resampled['whether he/she donated blood in March 2007']

Original class distribution: Counter({0: 570, 1: 178})


In [7]:
# Function to calculate the required metrics 
def get_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average=None)
    auc = roc_auc_score(y_true, y_prob[:, 1])
    return accuracy, precision, recall, f1, auc

# Define classifiers
classifiers = {
    'GaussianNB': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'RandomForest': RandomForestClassifier(random_state=42)
}

# Function to classify and store metrics
def classify_and_store(classifiers, X_train, y_train, X_test, y_test, oversampler=None):
    if oversampler:
        # Apply oversampling
        X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
    else:
        X_train_resampled, y_train_resampled = X_train, y_train
    
    # Store the metrics for each classifier
    classifier_metrics = {}
    
    for name, clf in classifiers.items():
        # Train the classifier
        clf.fit(X_train_resampled, y_train_resampled)
        
        # Predict on the test set
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)
        
        # Get metrics
        accuracy, precision, recall, f1, auc = get_metrics(y_test, y_pred, y_prob)
        classifier_metrics[name] = {
            'accuracy': accuracy,
            'precision_0': precision[0],
            'precision_1': precision[1],
            'recall_0': recall[0],
            'recall_1': recall[1],
            'f1_0': f1[0],
            'f1_1': f1[1],
            'auc_0': auc if len(np.unique(y_test)) == 2 else None,
            'auc_1': auc if len(np.unique(y_test)) == 2 else None
        }
    
    return classifier_metrics


# Split the dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(df_resampled, y_resampled, test_size=0.2, random_state=42)

# Standardizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 1: Perform baseline evaluation (without oversampling) and store results
oversamplers = {
    'RandomOverSampler': RandomOverSampler(random_state=42, sampling_strategy='minority'),
    'SMOTE': SMOTE(random_state=42, sampling_strategy='minority'),
    'ADASYN': ADASYN(random_state=42, sampling_strategy='minority'),
    'KMeansSMOTE': KMeansSMOTE(random_state=42, sampling_strategy='minority'),
    'SVMSMOTE': SVMSMOTE(random_state=42, sampling_strategy='minority')
}

# Store all results in a single DataFrame
all_results = []

# Step 2: Baseline (no oversampling)
baseline_metrics = classify_and_store(classifiers, X_train, y_train, X_test, y_test)
for clf_name, m in baseline_metrics.items():
    all_results.append({
        'Classifier': clf_name,
        'Oversampler': 'None',
        'Accuracy': m['accuracy'],
        'Precision(0)': m['precision_0'],
        'Precision(1)': m['precision_1'],
        'Recall (0)': m['recall_0'],
        'Recall (1)': m['recall_1'],
        'F1(0)': m['f1_0'],
        'F1(1)': m['f1_1'],
        'AUC(0)': m['auc_0'],
        'AUC(1)': m['auc_1']
    })

# Step 3: Loop over classifiers and then oversamplers
for clf_name, clf in classifiers.items():
    for sampler_name, oversampler in oversamplers.items():
        oversampled_metrics = classify_and_store({clf_name: clf}, X_train, y_train, X_test, y_test, oversampler)
        
        # Now m will be the dictionary with metrics for the classifier
        m = oversampled_metrics[clf_name]
        
        all_results.append({
            'Classifier': clf_name,
            'Oversampler': sampler_name,
            'Accuracy': m['accuracy'],
            'Precision(0)': m['precision_0'],
            'Precision(1)': m['precision_1'],
            'Recall (0)': m['recall_0'],
            'Recall (1)': m['recall_1'],
            'F1(0)': m['f1_0'],
            'F1(1)': m['f1_1'],
            'AUC(0)': m['auc_0'],
            'AUC(1)': m['auc_1']
        })

# Convert to DataFrame
results_df = pd.DataFrame(all_results)


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=3
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=3
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1955: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=3
  warnings.warn(


_______________________________________________________________________________________________________________________________________________________________________

In [8]:
transfusion[transfusion['whether he/she donated blood in March 2007']==1].corr().abs().sum().sort_values()

whether he/she donated blood in March 2007    0.000000
Recency (months)                              1.452802
Time (months)                                 2.551066
Frequency (times)                             2.931335
Monetary (c.c. blood)                         2.931335
dtype: float64

In [9]:
df_resampled2[df_resampled2['whether he/she donated blood in March 2007']==1].count()

Recency (months)                              178
Frequency (times)                             178
Monetary (c.c. blood)                         178
Time (months)                                 178
whether he/she donated blood in March 2007    178
dtype: int64

In [10]:
457-178

279

In [11]:
minority_samples = df_resampled2[df_resampled2['whether he/she donated blood in March 2007']==1]
random_values = minority_samples['Frequency (times)'].sample(n=279, replace=True)
random_values = random_values.reset_index()
random_values = pd.DataFrame(random_values)
del random_values['index']
random_values

,Frequency (times)
0,6
1,3
2,9
3,5
4,6
...,...
274,10
275,5
276,6
277,4


In [12]:
transfusion.columns

Index(['Recency (months)', 'Frequency (times)', 'Monetary (c.c. blood)',
       'Time (months)', 'whether he/she donated blood in March 2007'],
      dtype='object')

In [13]:
random_values['Recency (months)'] = np.nan
random_values['Monetary (c.c. blood)'] = np.nan
random_values['Time (months)'] = np.nan
random_values['whether he/she donated blood in March 2007'] = 1

In [14]:
random_values

,Frequency (times),Recency (months),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,6,NaN,NaN,NaN,1
1,3,NaN,NaN,NaN,1
2,9,NaN,NaN,NaN,1
3,5,NaN,NaN,NaN,1
4,6,NaN,NaN,NaN,1
...,...,...,...,...,...
274,10,NaN,NaN,NaN,1
275,5,NaN,NaN,NaN,1
276,6,NaN,NaN,NaN,1
277,4,NaN,NaN,NaN,1


In [15]:
def calculate_percentiles(nums):
    indexed_nums = [(num, i) for i, num in enumerate(nums)]
    sorted_nums = []
    for num_index in indexed_nums:
        inserted = False
        for i, sorted_num_index in enumerate(sorted_nums):
            if num_index[0] < sorted_num_index[0]:
                sorted_nums.insert(i, num_index)
                inserted = True
                break
        if not inserted:
            sorted_nums.append(num_index)
    length = len(sorted_nums)
    percentiles = [0] * length
    for i, num_index in enumerate(sorted_nums):
        original_index = num_index[1]
        percentile = ((i + 1) / length) * 100
        percentiles[original_index] = percentile
    return percentiles

def dataframe_to_percentiles(df):
    df_percentiles = df.copy()
    for column in df.columns:
        if pd.api.types.is_numeric_dtype(df[column]):
            df_percentiles[column] = calculate_percentiles(df[column].tolist())
    return df_percentiles


In [16]:
random_values['Frequency (times)'] = calculate_percentiles(random_values['Frequency (times)'])
random_values['Frequency (times)'] = 'P' + random_values['Frequency (times)'].round().astype(str,errors='ignore')

In [17]:
random_values['Frequency (times)'].unique()

array(['P49.0', 'P22.0', 'P70.0', 'P37.0', 'P29.0', 'P0.0', 'P10.0',
       'P50.0', 'P11.0', 'P65.0', 'P51.0', 'P75.0', 'P100.0', 'P88.0',
       'P30.0', 'P23.0', 'P82.0', 'P38.0', 'P61.0', 'P1.0', 'P90.0',
       'P39.0', 'P93.0', 'P98.0', 'P24.0', 'P62.0', 'P71.0', 'P96.0',
       'P66.0', 'P40.0', 'P99.0', 'P52.0', 'P86.0', 'P12.0', 'P31.0',
       'P41.0', 'P76.0', 'P25.0', 'P91.0', 'P13.0', 'P94.0', 'P42.0',
       'P53.0', 'P77.0', 'P95.0', 'P43.0', 'P14.0', 'P89.0', 'P2.0',
       'P72.0', 'P87.0', 'P26.0', 'P54.0', 'P3.0', 'P15.0', 'P67.0',
       'P32.0', 'P92.0', 'P78.0', 'P68.0', 'P44.0', 'P4.0', 'P83.0',
       'P16.0', 'P27.0', 'P63.0', 'P5.0', 'P97.0', 'P79.0', 'P55.0',
       'P45.0', 'P33.0', 'P46.0', 'P17.0', 'P84.0', 'P85.0', 'P56.0',
       'P18.0', 'P6.0', 'P7.0', 'P80.0', 'P34.0', 'P69.0', 'P47.0',
       'P19.0', 'P64.0', 'P57.0', 'P81.0', 'P48.0', 'P35.0', 'P73.0',
       'P20.0', 'P28.0', 'P8.0', 'P58.0', 'P9.0', 'P59.0', 'P74.0',
       'P36.0', 'P21.0', 'P60

In [18]:
transfusionP = dataframe_to_percentiles(minority_samples)
transfusionP = 'P' + transfusionP.round().astype(str,errors='ignore')
transfusionP['whether he/she donated blood in March 2007'] = 1

In [19]:
minority_samples

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
6,2,7,1750,14,1
...,...,...,...,...,...
680,20,14,3500,69,1
695,17,7,1750,58,1
708,11,2,500,38,1
712,14,2,500,35,1


In [20]:
ZERO = pd.concat([transfusionP,random_values], ignore_index=True)
ZERO

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,P3.0,P100.0,P100.0,P98.0,1
1,P1.0,P84.0,P84.0,P46.0,1
2,P2.0,P90.0,P90.0,P60.0,1
3,P3.0,P94.0,P94.0,P75.0,1
4,P4.0,P58.0,P58.0,P21.0,1
...,...,...,...,...,...
452,NaN,P81.0,NaN,NaN,1
453,NaN,P48.0,NaN,NaN,1
454,NaN,P61.0,NaN,NaN,1
455,NaN,P37.0,NaN,NaN,1


In [21]:


# Step 1: Identify unique values of 'pH'
unique_size = transfusionP['Frequency (times)'].unique()

# Step 2: Create a dictionary to store non-missing values for each variable
# Initialize the dictionary
imputation_info = {}

# Iterate over each variable (excluding 'pH') that has missing values
for column in transfusionP.columns:
    if column != 'Age' and ZERO[column].isna().sum() > 0:
        imputation_info[column] = {}
        
        # Iterate over each unique value of 'pH'
        for size_value in unique_size:
            # Get the non-missing values of the variable where 'pH' is equal to the current pH_value
            non_missing_values = transfusionP.loc[transfusionP['Frequency (times)'] == size_value, column].dropna().values
            imputation_info[column][size_value] = non_missing_values

In [22]:
# import random
import statistics
def fill_missing_values2(row, imputation_info):
    # For each column, check if it has a missing value
    for column in imputation_info:
        if pd.isna(row[column]):
            size_value = row['Frequency (times)']  # Get the corresponding pH value for the row
            if size_value in imputation_info[column]:
                possible_values = imputation_info[column][size_value]
                if len(possible_values) > 0:
                    row[column ] = statistics.mode(possible_values)
                    # row[column] = random.choice(possible_values)

    return row

myMode = ZERO.apply(lambda row: fill_missing_values2(row, imputation_info), axis=1)

In [23]:
myMode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457 entries, 0 to 456
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Recency (months)                            456 non-null    object
 1   Frequency (times)                           457 non-null    object
 2   Monetary (c.c. blood)                       456 non-null    object
 3   Time (months)                               456 non-null    object
 4   whether he/she donated blood in March 2007  457 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 18.0+ KB


In [24]:
myMode[myMode.isnull().any(axis=1)]['Frequency (times)'].value_counts()
myMode = myMode.fillna(myMode.iloc[0])

In [25]:
print(myMode.shape)
print(ZERO.shape)

(457, 5)
(457, 5)


In [26]:
percentiles = [*range(0,101, 1)]
mapping_data = {}

columns_to_impute = transfusion.columns.difference(['whether he/she donated blood in March 2007'])
for variable in columns_to_impute:
    mapping_data[variable] = [transfusion[transfusion['whether he/she donated blood in March 2007']==1][variable].quantile(p / 100) for p in percentiles]

# Create the mapping DataFrame
mapping_df = pd.DataFrame(mapping_data, index=[f'P{p}' for p in percentiles])
print("Mapping DataFrame:")
mapping_df

Mapping DataFrame:


,Frequency (times),Monetary (c.c. blood),Recency (months),Time (months)
P0,1.00,250.0,0.00,2.00
P1,1.00,250.0,0.77,2.00
P2,1.00,250.0,1.54,2.00
P3,1.00,250.0,2.00,2.31
P4,1.00,250.0,2.00,4.00
...,...,...,...,...
P96,21.92,5480.0,16.92,85.44
P97,30.83,7707.5,20.69,88.38
P98,37.22,9305.0,21.00,98.00
P99,43.69,10922.5,22.23,98.00


In [27]:
def convert_percentiles_to_values(mdf, mapping_df):
    columns_to_impute = mdf.columns.difference(['whether he/she donated blood in March 2007'])

    for column in columns_to_impute:
        mdf[column] = mdf[column].apply(lambda x: mapping_df.loc[x, column] if isinstance(x, str) and x.startswith('P') else x)
    return mdf

In [28]:
def path_to_revert(now, then):
    now = now.where(then.isna(), then)
    now = now.replace(r'^(P\d+)\.0$', r'\1', regex=True)
    return now

In [29]:
a = path_to_revert(myMode, ZERO)
a = convert_percentiles_to_values(a, mapping_df)

In [30]:
a

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2.00,50.00,12500.0,98.00,1
1,0.77,12.68,3170.0,28.00,1
2,1.54,16.00,4000.0,35.00,1
3,2.00,20.00,5000.0,44.50,1
4,2.00,6.66,1665.0,14.00,1
...,...,...,...,...,...
452,2.00,11.00,2750.0,41.21,1
453,4.00,5.00,1250.0,17.23,1
454,5.52,7.00,1750.0,26.00,1
455,3.34,4.49,1122.5,30.74,1


In [31]:
percent = pd.concat([a, df_resampled2[df_resampled2['whether he/she donated blood in March 2007']==0][0:457]], ignore_index=True)

In [32]:
percent['whether he/she donated blood in March 2007'].value_counts()

whether he/she donated blood in March 2007
1    457
0    457
Name: count, dtype: int64

In [33]:
X_percent = percent.drop(columns=['whether he/she donated blood in March 2007'])  # Replace with actual target column name
y_percent = percent['whether he/she donated blood in March 2007']  # Replace with actual target column name

# Standardize the 'percent' data as well
X_percent = scaler.fit_transform(X_percent)

def evaluate_custom_oversampling(X_train, y_train, X_test, y_test, classifiers):
    custom_results = []
    
    # Iterate over classifiers first
    for clf_name, clf in classifiers.items():
        # Train the classifier
        clf.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = clf.predict(X_test)
        y_prob = clf.predict_proba(X_test)
        
        # Get metrics
        accuracy, precision, recall, f1, auc = get_metrics(y_test, y_pred, y_prob)
        
        # Append the result
        custom_results.append({
            'Classifier': clf_name,               # Classifier first
            'Oversampler': 'PERCENTILES',              # Oversampler (Custom in this case)
            'Accuracy': accuracy,
            'Precision(0)': precision[0],
            'Precision(1)': precision[1],
            'Recall (0)': recall[0],
            'Recall (1)': recall[1],
            'F1(0)': f1[0],
            'F1(1)': f1[1],
            'AUC(0)': auc,
            'AUC(1)': auc
        })
    
    # Convert to DataFrame and return
    return pd.DataFrame(custom_results)



# Evaluate on custom oversampled data (the 'percent' dataset)
custom_results_df = evaluate_custom_oversampling(X_percent, y_percent, X_test, y_test, classifiers)


# Output: Print or review the custom oversampling results
custom_results_df

,Classifier,Oversampler,Accuracy,Precision(0),Precision(1),Recall (0),Recall (1),F1(0),F1(1),AUC(0),AUC(1)
0,GaussianNB,PERCENTILES,0.720000,0.793388,0.413793,0.849558,0.324324,0.820513,0.363636,0.698398,0.698398
1,KNN,PERCENTILES,0.680000,0.882353,0.415385,0.663717,0.729730,0.757576,0.529412,0.690385,0.690385
2,RandomForest,PERCENTILES,0.693333,0.819048,0.400000,0.761062,0.486486,0.788991,0.439024,0.643985,0.643985


In [35]:
combined_df = pd.concat([results_df, custom_results_df], ignore_index=True)

# Save to Excel
combined_df.to_excel("transfusion3.xlsx", index=False)
